In [18]:
#Importing libraries
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.base import TransformerMixin
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import requests
import seaborn as sns
import pprint, time
from sklearn.model_selection import train_test_split
nltk.download('treebank')
from nltk.tokenize import sent_tokenize
import string
from nltk.tokenize import RegexpTokenizer
from nltk.stem import WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from collections import Counter
from spellchecker import SpellChecker
import random
from sklearn_crfsuite import CRF
from sklearn_crfsuite import metrics
from sklearn_crfsuite import scorers
import base64
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

nltk.download('punkt')

nltk.download('universal_tagset')

%matplotlib inline

punct = string.punctuation

[nltk_data] Downloading package treebank to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package treebank is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package universal_tagset to
[nltk_data]     C:\Users\halapets\AppData\Roaming\nltk_data...
[nltk_data]   Package universal_tagset is already up-to-date!


In [19]:
#loading spacy
import spacy
nlp = spacy.load("en_core_web_lg")
nlp.max_length = 16000000

In [20]:
from spacy.lang.en.stop_words import STOP_WORDS
stopwords = list(STOP_WORDS)
stopwords

['amount',
 'two',
 'nobody',
 'him',
 'below',
 'ca',
 'under',
 'about',
 'we',
 '’s',
 'on',
 'they',
 'ten',
 'what',
 'had',
 'most',
 'too',
 'empty',
 'here',
 'another',
 'side',
 'own',
 '’ve',
 'same',
 'then',
 '‘ll',
 'so',
 'am',
 'thus',
 'since',
 'seeming',
 'twelve',
 'and',
 'take',
 'therefore',
 'per',
 'n‘t',
 'cannot',
 'only',
 'call',
 'regarding',
 'should',
 'but',
 'besides',
 'made',
 'were',
 'sometimes',
 'n’t',
 'while',
 '‘s',
 'over',
 'its',
 'hereby',
 'would',
 'hers',
 'just',
 'throughout',
 'will',
 'move',
 'anyone',
 'onto',
 'anything',
 'do',
 'something',
 'yourself',
 'behind',
 'rather',
 'everything',
 'along',
 'whatever',
 'one',
 'seem',
 'see',
 'except',
 'else',
 'latterly',
 'bottom',
 'the',
 'when',
 'as',
 'up',
 'get',
 'top',
 "'re",
 "'ve",
 'that',
 'among',
 'few',
 'our',
 'now',
 'toward',
 'some',
 'formerly',
 'latter',
 'nowhere',
 'may',
 'whereby',
 'used',
 'these',
 'less',
 'more',
 "'d",
 'at',
 'third',
 'those',

In [21]:
LatestMobileReviews1 = pd.read_csv('C:\\Users\\halapets\\Documents\\LJMU Research_Sentiment Analysis\\LatestMobileReviews.csv',encoding='latin1')
LatestMobileReviews1.head()

,Product,ReviewUpdated,Rating,Date,Website,TitleGoodBad,ProductPrice,ProductPriceRange,Year,positive_negative,...,Positive_Negative_Tag,LengthofReviews,SentimentPolarity,SentimentSubjectivity,SentimentPolarityClass,AvgSentPolarity,AvgSentimentPolarityClass,AvgSentimentSubjectivity,ReviewUpdated1,ReviewUpdated2
0,Xiaomi Redmi Note 3,"['yet', 'another', 'value', 'money', 'product'...",3.0,"Jul 17, 2016 10:59 PM",MouthShut,Yet another Value for Money product from Xiaomi,9999,Budget Range,2016.0,1,...,Positive Rating,344,-0.028571,0.576190,Negative Rating,0.403533,Positive Rating,NaN,Yet another Value for Money product from Xiaom...,Yet another Value for Money product from Xiaom...
1,Xiaomi Redmi Note 3,"['awesome', 'marvelous', 'mobile', 'go', 'valu...",5.0,"Jul 17, 2016 10:42 PM",MouthShut,Awesome,9999,Budget Range,2016.0,1,...,Positive Rating,276,0.780000,0.640000,Positive Rating,0.403533,Positive Rating,NaN,"Awesome Marvelous Mobile just go for it., Val...","Awesome Marvelous Mobile just go for it., Val..."
2,Xiaomi Redmi Note 3,"['power', 'pack', 'review', 'redmi', 'note', '...",5.0,"Jul 17, 2016 06:18 PM",MouthShut,Power pack review on Redmi Note 3,9999,Budget Range,2016.0,1,...,Positive Rating,322,0.474444,0.477037,Positive Rating,0.403533,Positive Rating,NaN,Power pack review on Redmi Note 3 REDMI NOTE ...,Power pack review on Redmi Note 3 REDMI NOTE ...
3,Xiaomi Redmi Note 3,"['flagship', 'smartfone', 'phone', 'looks', 'v...",4.0,"Jul 17, 2016 04:44 PM",MouthShut,Flagship smartfone under 13000rs,9999,Budget Range,2016.0,1,...,Positive Rating,333,0.502500,0.832500,Positive Rating,0.403533,Positive Rating,NaN,Flagship smartfone under 13000rs This phone l...,Flagship smartfone under 13000rs This phone l...
4,Xiaomi Redmi Note 3,"['boon', 'mobile', 'master', 'piece', 'economi...",5.0,"Jul 17, 2016 12:32 PM",MouthShut,Boon for mobile,9999,Budget Range,2016.0,1,...,Positive Rating,316,0.448571,0.627857,Positive Rating,0.403533,Positive Rating,NaN,Boon for mobile Master piece in such a econom...,Boon for mobile Master piece in such a econom...


In [45]:
#tokenization and text preprocessing
def text_data_cleaning(sentence):
    doc = nlp(sentence)
    
    tokens = []
    for token in doc:
        if token.lemma_ != "-PRON-" or token.pos_ != "AUX" or token.pos_ != "NUM" or token.pos_ != "SYM" or token.pos_ != "ADP"  :
            temp = token.lemma_.lower().strip()
        tokens.append(temp)
        
    
    cleaned_tokens = []
    for token in tokens:
        if token not in stopwords and token not in punct and token.isalpha():
            cleaned_tokens.append(token)
    return cleaned_tokens

In [46]:
text_data_cleaning("    Hello how are you. Like this video! Please advise $")
LatestMobileReviews1.columns

Index(['Product', 'ReviewUpdated', 'Rating', 'Date', 'Website', 'TitleGoodBad',
       'ProductPrice', 'ProductPriceRange', 'Year', 'positive_negative',
       'CountofReviews', 'AvgRating', 'AvgRatingScore',
       'Positive_Negative_Tag', 'LengthofReviews', 'SentimentPolarity',
       'SentimentSubjectivity', 'SentimentPolarityClass', 'AvgSentPolarity',
       'AvgSentimentPolarityClass', 'AvgSentimentSubjectivity',
       'ReviewUpdated1', 'ReviewUpdated2'],
      dtype='object')

### TF-IDF Vectorization and LinearSVC (Support Vector Classifier)

In [47]:
from sklearn.svm import LinearSVC#change

#from sklearn.model_selection import GridSearchCV 

In [48]:
#word vectorization using tfidf
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)####change
#create base classifier
classifier = LinearSVC()###change

In [49]:
X = LatestMobileReviews1[['Product','ReviewUpdated1']]
y= LatestMobileReviews1['positive_negative']

In [50]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.3, random_state = 1001)

In [51]:
X_train.shape, y_train.shape, X_test.shape, y_test.shape

((112039, 2), (112039,), (48018, 2), (48018,))

In [52]:
#creating a pipeline of steps to be followed for classifier
clf_TFIDF_SVM = Pipeline([('tfidf', tfidf), ('clf', classifier)])#change

In [53]:
#fitting model
clf_TFIDF_SVM.fit(X_train['ReviewUpdated1'], y_train,)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x0000017D927C8C18>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LinearSVC(C=1.0, class_weight=None, dual=True,
                           fit_inter

In [54]:
#Evaluating model on test data
y_pred_TFIDF_SVM = clf_TFIDF_SVM.predict(X_test['ReviewUpdated1'])


In [55]:
target_names = ['class 0', 'class 1']
ClassificationReport_TFIDF_SVM=classification_report(y_test, y_pred_TFIDF_SVM, target_names=target_names)
ClassificationReport_TFIDF_SVM

#'              precision    recall  f1-score   support\n\n     
#class 0       0.77      0.57      0.66      7791\n     
#class 1       0.92      0.97      0.94     40227\n\n    
#accuracy                          0.90     48018\n   
#macro avg     0.85      0.77      0.80     48018\n
#weighted avg  0.90      0.90      0.90     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.77      0.57      0.66      7791\n     class 1       0.92      0.97      0.94     40227\n\n    accuracy                           0.90     48018\n   macro avg       0.85      0.77      0.80     48018\nweighted avg       0.90      0.90      0.90     48018\n'

In [56]:
ConfusionMatrix_TFIDF_SVM=confusion_matrix(y_test, y_pred_TFIDF_SVM)
ConfusionMatrix_TFIDF_SVM

array([[ 4441,  3350],
       [ 1293, 38934]], dtype=int64)

In [57]:
accuracy_score(y_test, y_pred_TFIDF_SVM) 
#Accuracy-0.9033070931733933
#Precision-


0.9033070931733933

In [58]:
y_pred_TFIDF_SVM = pd.DataFrame(y_pred_TFIDF_SVM,X_test['Product'])
y_pred_TFIDF_SVM.rename(columns={0:"Tags_TFIDF_SVM"}, inplace = True)
y_pred_TFIDF_SVM.head(5)
#len(y_pred_TFIDF_SVM)
y_pred_TFIDF_SVM.to_csv("y_pred_TFIDF_SVM.csv",header = True)

In [112]:
# defining parameter range 
#param_grid = {'C': [0.1, 1, 10, 100, 1000]} 

In [113]:
# Instantiate the grid search model
#grid_search = GridSearchCV(classifier, param_grid,cv = 3, verbose = 1,n_jobs=1)

In [87]:
# Fit the grid search to the data
#grid_search.fit(tfidf.transform(X_train), y_train)

Fitting 3 folds for each of 5 candidates, totalling 15 fits


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\svm\_base.py:947: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
[Parallel(n_jobs=1)]: Done  15 out of  15 | elapsed:  4.0min finished


GridSearchCV(cv=3, error_score=nan,
             estimator=LinearSVC(C=1.0, class_weight=None, dual=True,
                                 fit_intercept=True, intercept_scaling=1,
                                 loss='squared_hinge', max_iter=1000,
                                 multi_class='ovr', penalty='l2',
                                 random_state=None, tol=0.0001, verbose=0),
             iid='deprecated', n_jobs=1,
             param_grid={'C': [0.1, 1, 10, 100, 1000]}, pre_dispatch='2*n_jobs',
             refit=True, return_train_score=False, scoring=None, verbose=1)

In [92]:
#best_grid = grid_search.best_estimator_
#best_grid

LinearSVC(C=1, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [96]:
#y_pred_grid_search = grid_search.predict(tfidf.transform(X_test))

In [114]:
#target_names = ['class 0', 'class 1']
#classification_report_SVM_gridsearch=classification_report(y_test, y_pred_grid_search, target_names=target_names)
#classification_report_SVM_gridsearch

'              precision    recall  f1-score   support\n\n     class 0       0.80      0.65      0.72     16505\n     class 1       0.91      0.96      0.93     61332\n\n    accuracy                           0.89     77837\n   macro avg       0.85      0.80      0.82     77837\nweighted avg       0.89      0.89      0.89     77837\n'

In [115]:
#ConfusionMatrix_SVM_gridsearch=confusion_matrix(y_test, y_pred_grid_search)
#ConfusionMatrix_SVM_gridsearch

array([[10708,  5797],
       [ 2737, 58595]], dtype=int64)

In [116]:
#accuracy_score(y_test, y_pred_grid_search) #0.8903606254095097 accuracy of base model

0.8903606254095097

### TF-IDF Vectorization and Logistic Regression Classifier


In [59]:
from sklearn import linear_model
#word vectorization using tfidf
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
#create base classifier
LR = linear_model.LogisticRegression()

#creating a pipeline of steps to be followed for classifier
clf_TFIDF_LR = Pipeline([('tfidf', tfidf), ('clf', LR)])#change

#fitting model
clf_TFIDF_LR.fit(X_train['ReviewUpdated1'], y_train)



C:\Users\halapets\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:939: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html.
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  extra_warning_msg=_LOGISTIC_SOLVER_CONVERGENCE_MSG)


Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...,
                                 tokenizer=<function text_data_cleaning at 0x0000017D927C8C18>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 LogisticRegression(C=1.0, class_weight=None, dual=False,
                                    fit_i

In [60]:
#Evaluating model on test data
y_pred_TFIDF_LR = clf_TFIDF_LR.predict(X_test['ReviewUpdated1'])


In [61]:
target_names = ['class 0', 'class 1']
ClassificationReport_TFIDF_LR=classification_report(y_test, y_pred_TFIDF_LR, target_names=target_names)
ClassificationReport_TFIDF_LR

#'              precision    recall  f1-score   support\n\n     
#class 0       0.81      0.54      0.65      7791\n     
#class 1       0.92      0.97      0.94     40227\n\n    
#accuracy                          0.90     48018\n   
#macro avg     0.86      0.76      0.80     48018\n
#weighted avg  0.90      0.90      0.90     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.81      0.54      0.65      7791\n     class 1       0.92      0.97      0.94     40227\n\n    accuracy                           0.90     48018\n   macro avg       0.86      0.76      0.80     48018\nweighted avg       0.90      0.90      0.90     48018\n'

In [62]:
ConfusionMatrix_TFIDF_LR=confusion_matrix(y_test, y_pred_TFIDF_LR)
ConfusionMatrix_TFIDF_LR

array([[ 4221,  3570],
       [ 1018, 39209]], dtype=int64)

In [63]:
accuracy_score(y_test, y_pred_TFIDF_LR) #0.9044524969802991 accuracy of base model

0.9044524969802991

In [64]:
y_pred_TFIDF_LR = pd.DataFrame(y_pred_TFIDF_LR,X_test['Product'])
y_pred_TFIDF_LR.rename(columns={0:"Tags_TFIDF_LR"}, inplace = True)
y_pred_TFIDF_LR.head(5)
#len(y_pred_TFIDF_SVM)
y_pred_TFIDF_LR.to_csv("y_pred_TFIDF_LR.csv",header = True)

In [65]:
#Create Hyperparameter Search Space 
# Create regularization penalty space
#penalty = ['l1', 'l2']

# Create regularization hyperparameter space
#C = np.logspace(0, 4, 10)

# Create hyperparameter options
#hyperparameters = dict(C=C, penalty=penalty)

In [66]:
# Create grid search using 5-fold cross validation
#grid_search_LR = GridSearchCV(classifier, hyperparameters, cv=5, verbose=0)

In [67]:
# Fit the grid search to the data
#grid_search_LR.fit(tfidf.transform(X_train), y_train)

In [68]:

#best_grid_LR = grid_search_LR.best_estimator_

#best_grid_LR

In [ ]:
#y_pred_grid_search_LR = grid_search_LR.predict(tfidf.transform(X_test))

In [ ]:
#target_names = ['class 0', 'class 1']
#ClassificationReport_LR_gridsearch=classification_report(y_test, y_pred_grid_search_LR, target_names=target_names)

In [ ]:
#ConfusionMatrix_LR_gridsearch=confusion_matrix(y_test, y_pred_grid_search_LR)

In [ ]:
#accuracy_score(y_test, y_pred_grid_search_LR) #0.8903606254095097 accuracy of base model

### TF-IDF Vectorization and Naive Bayes Classifier


In [69]:
from sklearn.naive_bayes import BernoulliNB
#word vectorization using tfidf
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
#create base classifier
NB = BernoulliNB()


#creating a pipeline of steps to be followed for classifier
clf_TFIDF_NB = Pipeline([('tfidf', tfidf), ('clf', NB)])#change

#fitting model
clf_TFIDF_NB.fit(X_train['ReviewUpdated1'], y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='(?u)\\b\\w\\w+\\b',
                                 tokenizer=<function text_data_cleaning at 0x0000017D927C8C18>,
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 BernoulliNB(alpha=1.0, binarize=0.0, class_prior=None,
                            

In [70]:
#Evaluating model on test data
y_pred_TFIDF_NB = clf_TFIDF_NB.predict(X_test['ReviewUpdated1'])


In [71]:
target_names = ['class 0', 'class 1']
ClassificationReport_TFIDF_NB=classification_report(y_test, y_pred_TFIDF_NB, target_names=target_names)
ClassificationReport_TFIDF_NB

#'              precision    recall  f1-score   support\n\n     
#class 0       0.72      0.50      0.59      7791\n     
#class 1       0.91      0.96      0.93     40227\n\n    
#accuracy                          0.89     48018\n   
#macro avg     0.82      0.73      0.76     48018\n
#weighted avg  0.88      0.89      0.88     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.72      0.50      0.59      7791\n     class 1       0.91      0.96      0.93     40227\n\n    accuracy                           0.89     48018\n   macro avg       0.82      0.73      0.76     48018\nweighted avg       0.88      0.89      0.88     48018\n'

In [72]:
ConfusionMatrix_TFIDF_NB=confusion_matrix(y_test, y_pred_TFIDF_NB)
ConfusionMatrix_TFIDF_NB

array([[ 3868,  3923],
       [ 1487, 38740]], dtype=int64)

In [73]:
accuracy_score(y_test, y_pred_TFIDF_NB) #0.8873339164479986 accuracy of base model

0.8873339164479986

In [74]:
y_pred_TFIDF_NB = pd.DataFrame(y_pred_TFIDF_NB,X_test['Product'])
y_pred_TFIDF_NB.rename(columns={0:"Tags_TFIDF_NB"}, inplace = True)
y_pred_TFIDF_NB.head(5)
#len(y_pred_TFIDF_NB)
y_pred_TFIDF_NB.to_csv("y_pred_TFIDF_NB.csv",header = True)

### TF-IDF Vectorization and Decision Tree Classifier

In [75]:
from sklearn.tree import DecisionTreeClassifier 
#from sklearn.datasets import make_classification
#word vectorization using tfidf
tfidf = TfidfVectorizer(tokenizer = text_data_cleaning)
#create base classifier
cart = DecisionTreeClassifier(criterion = "gini", 
            random_state = 100) 

#creating a pipeline of steps to be followed for classifier
clf_TFIDF_CART = Pipeline([('tfidf', tfidf), ('clf', cart)])#change

#fitting model
clf_TFIDF_CART.fit(X_train['ReviewUpdated1'], y_train)

Pipeline(memory=None,
         steps=[('tfidf',
                 TfidfVectorizer(analyzer='word', binary=False,
                                 decode_error='strict',
                                 dtype=<class 'numpy.float64'>,
                                 encoding='utf-8', input='content',
                                 lowercase=True, max_df=1.0, max_features=None,
                                 min_df=1, ngram_range=(1, 1), norm='l2',
                                 preprocessor=None, smooth_idf=True,
                                 stop_words=None, strip_accents=None,
                                 sublinear_tf=False,
                                 token_pattern='...
                                 use_idf=True, vocabulary=None)),
                ('clf',
                 DecisionTreeClassifier(ccp_alpha=0.0, class_weight=None,
                                        criterion='gini', max_depth=None,
                                        max_features=None, max_l

In [76]:
#Evaluating model on test data
y_pred_TFIDF_CART = clf_TFIDF_CART.predict(X_test['ReviewUpdated1'])


In [77]:
target_names = ['class 0', 'class 1']
ClassificationReport_TFIDF_CART=classification_report(y_test, y_pred_TFIDF_CART, target_names=target_names)
ClassificationReport_TFIDF_CART

#'              precision    recall  f1-score   support\n\n     
#class 0       0.53      0.49      0.51      7791\n     
#class 1       0.90      0.91      0.91     40227\n\n    
#accuracy                          0.85     48018\n   
#macro avg     0.71      0.70      0.71     48018\n
#weighted avg  0.84      0.85      0.84     48018\n'

'              precision    recall  f1-score   support\n\n     class 0       0.53      0.49      0.51      7791\n     class 1       0.90      0.91      0.91     40227\n\n    accuracy                           0.85     48018\n   macro avg       0.71      0.70      0.71     48018\nweighted avg       0.84      0.85      0.84     48018\n'

In [78]:
ConfusionMatrix_TFIDF_CART=confusion_matrix(y_test, y_pred_TFIDF_CART)
ConfusionMatrix_TFIDF_CART

array([[ 3795,  3996],
       [ 3429, 36798]], dtype=int64)

In [79]:
accuracy_score(y_test, y_pred_TFIDF_CART) #0.8453704860677246 accuracy of base model

0.8453704860677246

In [80]:
y_pred_TFIDF_CART = pd.DataFrame(y_pred_TFIDF_CART,X_test['Product'])
y_pred_TFIDF_CART.rename(columns={0:"Tags_TFIDF_CART"}, inplace = True)
y_pred_TFIDF_CART.head(5)
#len(y_pred_TFIDF_CART)
y_pred_TFIDF_CART.to_csv("y_pred_TFIDF_CART.csv",header = True)

In [ ]:
#random_grid = {'n_estimators': [200, 400, 600, 800, 1000, 1200, 1400, 1600, 1800, 2000],'max_features': [2,3]}

In [ ]:
# Create grid search using 5-fold cross validation
#rf_random  = RandomizedSearchCV(rf, random_grid,n_iter = 100, cv=5, verbose=2, random_state=42, n_jobs = -1)

In [ ]:
# Fit the grid search to the data
#rf_random.fit(tfidf.transform(X_train), y_train)

In [ ]:

#best_grid_RF = rf_random.best_params_

#best_grid_RF

In [ ]:
#y_pred_grid_search_RF = rf_random.predict(tfidf.transform(X_test))

In [ ]:
#target_names = ['class 0', 'class 1']
#ClassificationReport_RF_gridsearch=classification_report(y_test, y_pred_grid_search_RF, target_names=target_names)

In [ ]:
#ConfusionMatrix_RF_gridsearch=confusion_matrix(y_test, y_pred_grid_search_RF)

In [ ]:
#accuracy_score(y_test, y_pred_grid_search_RF) #0.8903606254095097 accuracy of base model

In [ ]:
#Ensemble Model based on Individual Classifiers using Majority of voting

In [81]:
y_pred_TFIDF_NB = pd.read_csv("y_pred_TFIDF_NB.csv",index_col=None)
y_pred_TFIDF_LR= pd.read_csv("y_pred_TFIDF_LR.csv",index_col=None)
y_pred_TFIDF_CART= pd.read_csv("y_pred_TFIDF_CART.csv",index_col=None)
y_pred_TFIDF_SVM= pd.read_csv("y_pred_TFIDF_SVM.csv",index_col=None)

In [82]:
y_pred_TFIDF_ensb=y_pred_TFIDF_NB.merge(y_pred_TFIDF_LR, how = 'inner',left_index = True, right_index = True, on = 'Product')


In [83]:
y_pred_TFIDF_ensb = y_pred_TFIDF_ensb.merge(y_pred_TFIDF_CART, how='inner', left_index = True, right_index = True, on = 'Product')

In [84]:
y_pred_TFIDF_ensb.shape

(48018, 4)

In [85]:

y_pred_TFIDF_ensb.head()

,Product,Tags_TFIDF_NB,Tags_TFIDF_LR,Tags_TFIDF_CART
0,Xiaomi Redmi 4A,1,1,1
1,Lenovo K3 Note,1,1,0
2,Samsung Galaxy Note 3,1,1,1
3,Xiaomi Redmi Note 3,1,1,1
4,Xiaomi Redmi 3S Prime,1,1,1


In [86]:
y_pred_TFIDF_ensb = y_pred_TFIDF_ensb.merge(y_pred_TFIDF_SVM, how='inner', left_index = True, right_index = True, on = 'Product')

In [87]:

y_pred_TFIDF_ensb.columns

Index(['Product', 'Tags_TFIDF_NB', 'Tags_TFIDF_LR', 'Tags_TFIDF_CART',
       'Tags_TFIDF_SVM'],
      dtype='object')

In [88]:
#Adding weights on classifiers based on the performance of the classifiers in terms of accuracy, precision and recall
#Naive Bayes and Decision Tree has been added a weight of 0.5 each
y_pred_TFIDF_ensb["Total Votes"] = (0.5*y_pred_TFIDF_ensb["Tags_TFIDF_NB"])+y_pred_TFIDF_ensb["Tags_TFIDF_LR"] + (0.5*y_pred_TFIDF_ensb["Tags_TFIDF_CART"])+y_pred_TFIDF_ensb["Tags_TFIDF_SVM"]


In [89]:
y_pred_TFIDF_ensb["Majority of Votes"]  = [1 if x >=2 else 0 for x in y_pred_TFIDF_ensb["Total Votes"]] 

In [90]:
target_names = ['class 0', 'class 1']
ClassificationReport_TFIDF_Ensemble=classification_report(y_test, y_pred_TFIDF_ensb["Majority of Votes"], target_names=target_names)
ClassificationReport_TFIDF_Ensemble

#'              precision    recall  f1-score   support\n\n     
#class 0       0.79      0.57      0.66      7791\n     
#class 1       0.92      0.97      0.95     40227\n\n    
#accuracy                          0.91     48018\n   
#macro avg     0.85      0.77      0.80     48018\n
#weighted avg  0.90      0.91      0.90     48018\n'



'              precision    recall  f1-score   support\n\n     class 0       0.79      0.57      0.66      7791\n     class 1       0.92      0.97      0.95     40227\n\n    accuracy                           0.91     48018\n   macro avg       0.85      0.77      0.80     48018\nweighted avg       0.90      0.91      0.90     48018\n'

In [91]:
ConfusionMatrix_TFIDF_Ensemble=confusion_matrix(y_test, y_pred_TFIDF_ensb["Majority of Votes"])
ConfusionMatrix_TFIDF_Ensemble

array([[ 4467,  3324],
       [ 1206, 39021]], dtype=int64)

In [92]:
accuracy_score(y_test, y_pred_TFIDF_ensb["Majority of Votes"]) #0.9056603773584906 accuracy of base model

0.9056603773584906